# Performing EDA on NBA stats and awards data

### Importing packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)


### Importing original data

In [43]:
# Awards
mvp_df = pd.read_csv("MVP_Data/mvp_awards.csv", index_col=[0])
roy_df = pd.read_csv("MVP_Data/roy_awards.csv", index_col=[0])
dpoy_df = pd.read_csv("MVP_Data/dpoy_awards.csv", index_col=[0])
smoy_df = pd.read_csv("MVP_Data/smoy_awards.csv", index_col=[0])

# Stats
ppg_df = pd.read_csv("NBA_Stats/ppg_data.csv", index_col=[0])
advanced_stats_df = pd.read_csv("NBA_Stats/advanced_stats_data.csv", index_col=[0])
team_standings_df = pd.read_csv("NBA_Stats/team_standings.csv", index_col=[0])

In [3]:
df_list = [mvp_df, 
              roy_df,
              dpoy_df,
              smoy_df, 
              ppg_df, 
              advanced_stats_df,
              team_standings_df]

df_names = ["MVP",
            "ROY",
            "DPOY",
            "SMOY",
            "PPG",
            "ADVANCED_STATS",
            "TEAM_STANDINGS"]

for i in range(len(df_list)):
    print(f"{df_names[i]} dataframe contains: {df_list[i].shape[0]} rows & {df_list[i].shape[1]} columns\n")


MVP dataframe contains: 339 rows & 21 columns

ROY dataframe contains: 183 rows & 21 columns

DPOY dataframe contains: 400 rows & 24 columns

SMOY dataframe contains: 354 rows & 21 columns

PPG dataframe contains: 14423 rows & 31 columns

ADVANCED_STATS dataframe contains: 14423 rows & 28 columns

TEAM_STANDINGS dataframe contains: 715 rows & 29 columns



In [4]:
for df in df_list:
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 339 entries, 0 to 338
Data columns (total 21 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Rank     339 non-null    object 
 1   Player   339 non-null    object 
 2   Age      339 non-null    int64  
 3   Tm       339 non-null    object 
 4   First    339 non-null    float64
 5   Pts Won  339 non-null    float64
 6   Pts Max  339 non-null    int64  
 7   Share    339 non-null    float64
 8   G        339 non-null    int64  
 9   MP       339 non-null    float64
 10  PTS      339 non-null    float64
 11  TRB      339 non-null    float64
 12  AST      339 non-null    float64
 13  STL      339 non-null    float64
 14  BLK      339 non-null    float64
 15  FG%      339 non-null    float64
 16  3P%      330 non-null    float64
 17  FT%      339 non-null    float64
 18  WS       339 non-null    float64
 19  WS/48    339 non-null    float64
 20  Year     339 non-null    int64  
dtypes: float64(14), 

In [5]:
for df in df_list:
    print(round(df.isna().sum()/len(df)*100, 2))

Rank       0.00
Player     0.00
Age        0.00
Tm         0.00
First      0.00
Pts Won    0.00
Pts Max    0.00
Share      0.00
G          0.00
MP         0.00
PTS        0.00
TRB        0.00
AST        0.00
STL        0.00
BLK        0.00
FG%        0.00
3P%        2.65
FT%        0.00
WS         0.00
WS/48      0.00
Year       0.00
dtype: float64
Rank       0.00
Player     0.00
Age        0.00
Tm         0.00
First      0.00
Pts Won    0.00
Pts Max    0.00
Share      0.00
G          0.00
MP         0.00
PTS        0.00
TRB        0.00
AST        0.00
STL        0.00
BLK        0.00
FG%        0.00
3P%        3.28
FT%        0.00
WS         0.00
WS/48      0.00
Year       0.00
dtype: float64
Rank       0.00
Player     0.00
Age        0.00
Tm         0.00
First      0.00
Pts Won    0.00
Pts Max    0.00
Share      0.00
G          0.00
MP         0.00
PTS        0.00
TRB        0.00
AST        0.00
STL        0.00
BLK        0.00
FG%        0.00
3P%        7.75
FT%        0.00
WS        

### Performing data pre-processing

#### 1. Merging players dataframe with the advanced stats table

In [24]:
players_df = ppg_df.merge(advanced_stats_df, 
                                     how='left', 
                                     left_on=['Player', 'Year', 'Tm'], 
                                     right_on=['Player', 'Year', 'Tm'],
                                     suffixes=('','_remove'))

players_df.drop([i for i in players_df.columns if "remove" in i], axis=1, inplace=True)

players_df = players_df[['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM',
       'BPM', 'VORP', 'Year']]

#### 2. Merging players + advanced stats dataframe with the team standings table

In [7]:
# Creating functions to rename NBA teams that have updated their team names to be consistent across both tables
def rename_teams_short(col):
    if col == 'CHO' or col == 'CHH':
        return "CHA"
    if col == "NOH" or col == "NOK":
        return "NOP"
    if col == 'TOT':
        return "DEN"
    else:
        return col

def rename_teams_long(col):
    if col == "Charlotte Bobcats":
        return "Charlotte Hornets"
    if col == "New Orleans Hornets" or col == "New Orleans/Oklahoma City Hornets":
        return "New Orleans Pelicans"
    else:
        return col

In [25]:
team_standings_df['Team'] = team_standings_df['Team'].str.replace('*','')

players_df['Tm'] = players_df['Tm'].apply(rename_teams_short)
team_standings_df['Team'] = team_standings_df['Team'].apply(rename_teams_long)


team_long = team_standings_df.Team.unique()
team_long.sort()
team_long = list(team_long)

team_short = players_df['Tm'].unique()
team_short.sort()
team_short = list(team_short)

team_dict = dict(zip(team_short, team_long))

players_df['Team_Long'] = players_df['Tm'].map(team_dict)

C:\Users\Nguye\AppData\Local\Temp\ipykernel_19924\406721146.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  team_standings_df['Team'] = team_standings_df['Team'].str.replace('*','')


In [26]:
team_standings_filtered_df = team_standings_df[['Team', 'W', 'L', 'MOV', 'SOS', 'SRS', 'ORtg',
       'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'eFG%', 'TOV%', 'ORB%',
       'FT/FGA', 'eFG%.1', 'TOV%.1', 'DRB%', 'FT/FGA.1', 'Year']]


team_standings_filtered_df = team_standings_filtered_df.add_prefix('Team_')
team_standings_filtered_df.rename(columns={"Team_Team":"Team", "Team_Year":"Year"}, inplace=True)


In [27]:
team_standings_filtered_df

,Team,Team_W,Team_L,Team_MOV,Team_SOS,Team_SRS,Team_ORtg,Team_DRtg,Team_NRtg,Team_Pace,Team_FTr,Team_3PAr,Team_TS%,Team_eFG%,Team_TOV%,Team_ORB%,Team_FT/FGA,Team_eFG%.1,Team_TOV%.1,Team_DRB%,Team_FT/FGA.1,Year
0,Los Angeles Lakers,67,15,8.55,-0.14,8.41,107.3,98.2,9.1,93.3,0.346,0.153,0.525,0.484,12.7,30.6,0.241,0.443,13.4,73.1,0.222,2000
1,Portland Trail Blazers,59,23,6.40,-0.04,6.36,107.9,100.8,7.1,89.9,0.316,0.175,0.546,0.501,14.5,30.3,0.240,0.461,13.8,72.4,0.217,2000
2,San Antonio Spurs,53,29,5.94,-0.02,5.92,105.0,98.6,6.4,90.8,0.346,0.138,0.535,0.488,14.3,27.8,0.258,0.451,13.5,73.0,0.188,2000
3,Phoenix Suns,53,29,5.22,0.02,5.24,104.6,99.0,5.6,94.0,0.286,0.184,0.532,0.491,15.2,29.3,0.217,0.454,15.7,70.5,0.245,2000
4,Utah Jazz,55,27,4.46,0.05,4.52,107.3,102.3,5.0,89.6,0.337,0.134,0.540,0.490,14.3,29.5,0.260,0.477,15.0,73.2,0.256,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710,Portland Trail Blazers,33,49,-4.01,0.05,-3.96,114.8,118.8,-4.0,98.6,0.289,0.413,0.589,0.549,13.1,22.4,0.230,0.563,12.1,74.9,0.217,2023
711,Charlotte Hornets,27,55,-6.24,0.35,-5.89,109.2,115.3,-6.1,100.8,0.261,0.360,0.550,0.516,12.3,23.8,0.195,0.544,12.5,75.5,0.211,2023
712,Houston Rockets,22,60,-7.85,0.24,-7.62,111.4,119.3,-7.9,99.0,0.285,0.359,0.554,0.516,14.0,30.2,0.215,0.564,11.8,75.8,0.218,2023
713,Detroit Pistons,17,65,-8.22,0.49,-7.73,110.7,118.9,-8.2,99.0,0.295,0.372,0.561,0.520,13.3,24.9,0.227,0.557,11.9,74.0,0.231,2023


In [28]:
players_df = players_df.merge(team_standings_filtered_df, 
                 how='left',
                 left_on=['Team_Long', 'Year'],
                 right_on=['Team', 'Year'])

# Some players have "*" so we want to remove it so we can merge the dfs
players_df['Player'] = players_df['Player'].str.replace('*','')

players_df['Total_Games'] = players_df['Team_W'] + players_df['Team_L']
players_df['Team_Win_PCT'] = round(players_df['Team_W'] / players_df['Total_Games'], 2)

players_df = players_df[['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER', 'TS%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP',
        'Team', 'Total_Games', 'Team_W', 'Team_L', 'Team_Win_PCT', 'Team_MOV', 'Team_SOS',
       'Team_SRS', 'Team_ORtg', 'Team_DRtg', 'Team_NRtg', 'Team_Pace',
       'Team_FTr', 'Team_3PAr', 'Team_TS%', 'Team_eFG%', 'Team_TOV%',
       'Team_ORB%', 'Team_FT/FGA', 'Team_eFG%.1', 'Team_TOV%.1', 'Team_DRB%',
       'Team_FT/FGA.1', 'Year']]


C:\Users\Nguye\AppData\Local\Temp\ipykernel_19924\2235533121.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  players_df['Player'] = players_df['Player'].str.replace('*','')


### 3. Cleaning NBA award tables

In [52]:
# Removing 'T' from the rank column
mvp_df['Rank'] = mvp_df['Rank'].str.split('T').apply(lambda x:x[0])
dpoy_df['Rank'] = dpoy_df['Rank'].str.split('T').apply(lambda x:x[0])
roy_df['Rank'] = dpoy_df['Rank'].str.split('T').apply(lambda x:x[0])
smoy_df['Rank'] = dpoy_df['Rank'].str.split('T').apply(lambda x:x[0])

mvp_df['Rank'] = mvp_df['Rank'].astype('Int32')
dpoy_df['Rank'] = dpoy_df['Rank'].astype('Int32')
roy_df['Rank'] = roy_df['Rank'].astype('Int32')
smoy_df['Rank'] = smoy_df['Rank'].astype('Int32')

#### Finding the minimum criterias to be considered an award candidate
- By finding the minimum criterias, we can reduce the dataset to only players that can be considered a potential award candidate which should help with training the model and balance the data by removing majority of players / stats that wouldn't qualify for the award since only a handful of players are nominated

#### Minimum Criteria for MVP winner:
1. Lowest Games played: 62 games 
2. Lowest Minutes Played: 30 minutes
3. Lowest PPG: 15.5 Pts

#### Minimum Criteria for top 5 MVP candidates:
1. Lowest Games played: 51 games
2. Lowest Minutes Played: 30 minutes
3. Lowest PPG: 12.6 Pts

Note: Looking at stats for each MVP winner and find the minimum MP, Games played, and Pts per game. However, we will also take a look at the minimum stats to qualify as a top 5 MVP candidate since those are the candidates more likely to win

In [68]:
round(mvp_df[mvp_df['Rank'] == 1].describe(),2)

,Rank,Age,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
count,24.0,24.00,24.00,24.00,24.00,24.00,24.00,24.00,24.00,24.00,24.00,24.00,24.00,24.00,24.00,24.00,24.00,24.00,24.00
mean,1.0,26.50,90.71,1074.50,1174.17,0.92,76.04,36.54,27.07,8.69,6.66,1.38,1.04,0.51,0.33,0.81,15.50,0.27,2011.50
std,0.0,2.06,23.09,131.80,113.40,0.07,5.98,2.91,4.12,3.66,2.29,0.52,0.84,0.05,0.10,0.09,2.81,0.04,7.07
min,1.0,22.00,57.00,875.00,1000.00,0.74,62.00,30.40,15.50,3.30,3.40,0.50,0.10,0.42,0.00,0.52,10.90,0.19,2000.00
25%,1.0,25.00,72.00,960.00,1010.00,0.88,72.00,34.52,24.90,5.40,4.90,1.00,0.48,0.49,0.32,0.77,13.10,0.24,2005.75
50%,1.0,26.50,85.50,1087.00,1210.00,0.93,78.50,36.80,27.40,7.95,6.45,1.40,0.80,0.50,0.34,0.83,15.50,0.28,2011.50
75%,1.0,28.00,113.75,1199.75,1252.50,0.98,81.00,38.92,29.80,12.55,8.00,1.72,1.20,0.55,0.39,0.87,18.00,0.30,2017.25
max,1.0,31.00,131.00,1310.00,1310.00,1.00,82.00,42.00,33.10,13.90,11.50,2.50,3.00,0.58,0.45,0.92,20.30,0.32,2023.00


In [82]:
round(mvp_df[mvp_df['Rank'] < 6].describe(),2)

,Rank,Age,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
count,120.0,120.00,120.00,120.00,120.00,120.00,120.00,120.00,120.00,120.00,120.00,120.00,120.00,120.00,118.00,120.00,120.00,120.00,120.00
mean,3.0,26.74,23.43,572.79,1174.17,0.49,74.39,36.56,25.95,8.18,5.80,1.38,1.03,0.50,0.31,0.79,13.18,0.23,2011.50
std,1.42,3.28,36.68,338.87,111.48,0.29,6.87,2.67,4.40,3.07,2.59,0.49,0.84,0.05,0.12,0.10,2.87,0.04,6.95
min,1.0,20.00,0.00,46.00,1000.00,0.05,51.00,30.40,12.60,2.70,1.30,0.50,0.10,0.39,0.00,0.46,6.20,0.13,2000.00
25%,2.0,24.00,0.00,306.25,1010.00,0.26,69.00,34.48,23.28,5.50,3.70,1.00,0.40,0.46,0.30,0.74,11.20,0.20,2005.75
50%,3.0,27.00,3.50,521.00,1210.00,0.43,76.00,36.45,26.80,7.85,5.50,1.40,0.75,0.49,0.34,0.81,13.05,0.23,2011.50
75%,4.0,29.00,25.25,878.25,1252.50,0.73,80.00,38.60,28.70,10.72,7.50,1.70,1.42,0.52,0.38,0.87,15.20,0.26,2017.25
max,5.0,36.00,131.00,1310.00,1310.00,1.00,82.00,42.50,36.10,14.20,11.60,2.80,3.70,0.63,0.46,0.93,20.30,0.32,2023.00


In [83]:
mvp_df[mvp_df['PTS'] < 15]

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
10,11,Jason Kidd,26,PHO,0.0,25.0,1210,0.021,67,39.0,14.3,7.2,10.1,2.0,0.4,0.409,0.337,0.829,8.5,0.156,2000
25,10,David Robinson,35,SAS,0.0,8.0,1240,0.006,80,29.6,14.4,8.6,1.5,1.0,2.5,0.486,0.000,0.747,12.1,0.246,2001
32,15,John Stockton,38,UTA,0.0,1.0,1240,0.001,82,29.2,11.5,2.8,8.7,1.6,0.3,0.504,0.462,0.817,10.8,0.216,2001
34,2,Jason Kidd,28,NJN,45.0,897.0,1260,0.712,82,37.3,14.7,7.3,9.9,2.1,0.2,0.391,0.321,0.814,8.9,0.140,2002
42,10,Ben Wallace,27,DET,0.0,24.0,1260,0.019,80,36.5,7.6,13.0,1.4,1.7,3.5,0.531,0.000,0.423,11.6,0.190,2002
48,16,Mike Bibby,23,SAC,0.0,1.0,1260,0.001,80,33.2,13.7,2.8,5.0,1.1,0.2,0.453,0.370,0.803,6.3,0.113,2002
58,8,Ben Wallace,28,DET,1.0,33.0,1190,0.028,73,39.4,6.9,15.4,1.6,1.4,3.2,0.481,0.167,0.450,10.6,0.176,2003
70,7,Ben Wallace,29,DET,0.0,24.0,1230,0.020,81,37.7,9.5,12.4,1.7,1.8,3.0,0.421,0.125,0.490,10.2,0.160,2004
93,14,P.J. Brown,35,NOH,0.0,1.0,1270,0.001,82,34.4,10.8,9.0,2.2,0.9,0.6,0.446,NaN,0.864,6.9,0.118,2005
94,14,Marcus Camby,30,DEN,0.0,1.0,1270,0.001,66,30.5,10.3,10.0,2.3,0.9,3.0,0.465,0.000,0.723,5.8,0.138,2005


In [84]:
mvp_df[mvp_df['Year'] == 2002]

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
33,1,Tim Duncan,25,SAS,57.0,954.0,1260,0.757,82,40.6,25.5,12.7,3.7,0.7,2.5,0.508,0.100,0.799,17.8,0.257,2002
34,2,Jason Kidd,28,NJN,45.0,897.0,1260,0.712,82,37.3,14.7,7.3,9.9,2.1,0.2,0.391,0.321,0.814,8.9,0.140,2002
35,3,Shaquille O'Neal,29,LAL,15.0,696.0,1260,0.552,67,36.1,27.2,10.7,3.0,0.6,2.0,0.579,0.000,0.555,13.2,0.262,2002
36,4,Tracy McGrady,22,ORL,7.0,390.0,1260,0.310,76,38.3,25.6,7.9,5.3,1.6,1.0,0.451,0.364,0.748,11.5,0.189,2002
37,5,Kobe Bryant,23,LAL,1.0,98.0,1260,0.078,80,38.3,25.2,5.5,5.5,1.5,0.4,0.469,0.250,0.829,12.7,0.199,2002
38,6,Gary Payton,33,SEA,1.0,54.0,1260,0.043,82,40.3,22.1,4.8,9.0,1.6,0.3,0.467,0.314,0.797,12.6,0.183,2002
39,7,Chris Webber,28,SAC,0.0,37.0,1260,0.029,54,38.4,24.5,10.1,4.8,1.7,1.4,0.495,0.263,0.749,8.6,0.200,2002
40,8,Dirk Nowitzki,23,DAL,0.0,31.0,1260,0.025,76,38.0,23.4,9.9,2.4,1.1,1.0,0.477,0.397,0.853,13.4,0.222,2002
41,9,Allen Iverson,26,PHI,0.0,29.0,1260,0.023,60,43.7,31.4,4.5,5.5,2.8,0.2,0.398,0.291,0.812,6.9,0.126,2002
42,10,Ben Wallace,27,DET,0.0,24.0,1260,0.019,80,36.5,7.6,13.0,1.4,1.7,3.5,0.531,0.000,0.423,11.6,0.190,2002


### There are instances where a player was traded to another team during the NBA season which results in their name appearing multiple instances. We can figure out how to fix this

### Creating a dataframe for MVP data

In [101]:
# Reducing columns that's unnecessary
mvp_subset = mvp_df[['Player', 'Tm','Share', 'Year']].copy()

mvp_players_df = players_df.merge(mvp_subset, 
                 how='left',
                 left_on=['Player', 'Year', 'Tm'],
                 right_on=['Player', 'Year', 'Tm'])

mvp_players_df['Share'] = mvp_players_df['Share'].fillna(0)

mvp_players_df

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Year,Team_Long,Team,Team_W,Team_L,Team_MOV,Team_SOS,Team_SRS,Team_ORtg,Team_DRtg,Team_NRtg,Team_Pace,Team_FTr,Team_3PAr,Team_TS%,Team_eFG%,Team_TOV%,Team_ORB%,Team_FT/FGA,Team_eFG%.1,Team_TOV%.1,Team_DRB%,Team_FT/FGA.1,Share
0,Tariq Abdul-Wahad,SG,25,DEN,61,56,25.9,4.5,10.6,0.424,0.0,0.4,0.130,4.4,10.2,0.435,0.426,2.4,3.2,0.756,1.7,3.1,4.8,1.6,1.0,0.5,1.7,2.4,11.4,13.6,0.477,0.036,0.299,7.0,13.1,10.0,10.0,1.8,1.2,12.7,22.5,0.4,1.8,2.2,0.068,-1.2,-0.1,-1.2,0.3,2000,Denver Nuggets,Denver Nuggets,35,47,-2.12,0.36,-1.76,103.5,105.8,-2.3,94.7,0.306,0.202,0.517,0.476,14.0,29.2,0.222,0.479,13.0,71.6,0.235,0.0
1,Tariq Abdul-Wahad,SG,25,ORL,46,46,26.2,4.8,11.2,0.433,0.0,0.5,0.095,4.8,10.7,0.447,0.435,2.5,3.3,0.762,1.7,3.5,5.2,1.6,1.2,0.3,1.9,2.5,12.2,14.4,0.484,0.041,0.293,7.0,14.6,10.8,9.7,2.2,0.9,13.0,23.3,0.4,1.6,2.0,0.082,-1.0,0.2,-0.8,0.4,2000,Orlando Magic,Orlando Magic,41,41,0.68,-0.26,0.43,102.4,101.7,0.7,97.3,0.305,0.124,0.516,0.473,15.4,31.6,0.224,0.476,15.9,69.9,0.226,0.0
2,Tariq Abdul-Wahad,SG,25,DEN,15,10,24.9,3.4,8.7,0.389,0.1,0.1,0.500,3.3,8.6,0.388,0.393,2.1,2.8,0.738,1.6,1.9,3.5,1.7,0.4,0.8,1.3,2.1,8.9,10.8,0.448,0.015,0.321,6.9,8.2,7.6,11.0,0.8,2.2,11.3,19.7,0.0,0.2,0.2,0.023,-1.7,-0.9,-2.5,0.0,2000,Denver Nuggets,Denver Nuggets,35,47,-2.12,0.36,-1.76,103.5,105.8,-2.3,94.7,0.306,0.202,0.517,0.476,14.0,29.2,0.222,0.479,13.0,71.6,0.235,0.0
3,Shareef Abdur-Rahim,SF,23,VAN,82,82,39.3,7.2,15.6,0.465,0.4,1.2,0.302,6.9,14.4,0.478,0.477,5.4,6.7,0.809,2.7,7.4,10.1,3.3,1.1,1.1,3.0,3.0,20.3,20.2,0.547,0.075,0.431,8.0,22.7,15.3,15.5,1.5,1.9,14.1,25.0,6.2,2.6,8.8,0.132,2.6,-0.4,2.2,3.4,2000,Vancouver Grizzlies,Vancouver Grizzlies,22,60,-5.62,0.52,-5.10,102.3,108.5,-6.2,91.0,0.320,0.139,0.524,0.474,15.8,30.1,0.247,0.502,14.1,70.6,0.231,0.0
4,Cory Alexander,PG,26,DEN,29,2,11.3,1.0,3.4,0.286,0.3,1.2,0.257,0.7,2.2,0.302,0.332,0.6,0.8,0.773,0.3,1.2,1.4,2.0,0.8,0.1,1.0,1.3,2.8,8.8,0.381,0.357,0.224,2.6,11.3,6.9,25.7,3.7,0.4,20.6,18.0,-0.5,0.4,-0.1,-0.012,-4.1,1.8,-2.4,0.0,2000,Denver Nuggets,Denver Nuggets,35,47,-2.12,0.36,-1.76,103.5,105.8,-2.3,94.7,0.306,0.202,0.517,0.476,14.0,29.2,0.222,0.479,13.0,71.6,0.235,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14418,Thaddeus Young,PF,34,TOR,54,9,14.7,2.0,3.7,0.545,0.1,0.6,0.176,1.9,3.0,0.622,0.561,0.3,0.5,0.692,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4,14.1,0.573,0.172,0.131,9.4,14.6,11.8,12.9,3.4,0.6,16.7,13.5,0.7,1.1,1.8,0.109,-1.8,1.9,0.1,0.4,2023,Toronto Raptors,Toronto Raptors,41,41,1.48,0.12,1.59,115.5,114.0,1.5,97.1,0.257,0.351,0.555,0.517,10.3,27.8,0.201,0.565,15.3,76.7,0.223,0.0
14419,Trae Young,PG,24,ATL,73,73,34.8,8.2,19.0,0.429,2.1,6.3,0.335,6.1,12.7,0.476,0.485,7.8,8.8,0.886,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2,22.0,0.573,0.331,0.460,2.4,7.0,4.7,42.5,1.5,0.3,15.2,32.6,5.3,1.4,6.7,0.126,5.3,-2.0,3.3,3.4,2023,Atlanta Hawks,Atlanta Hawks,41,41,0.29,0.02,0.32,116.6,116.3,0.3,100.7,0.244,0.331,0.579,0.541,11.2,25.1,0.200,0.552,12.4,75.8,0.206,0.0
14420,Omer Yurtseven,C,24,MIA,9,0,9.2,1.8,3.0,0.593,0.3,0.8,0.429,1.4,2.2,0.650,0.648,0.6,0.7,0.833,0.9,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4,16.7,0.675,0.259,0.222,10.9,21.9,16.2,3.9,1.2,2.5,11.9,18.0,0.2,0.1,0.3,0.159,-2.5,-1.5,-3.9,0.0,2023,Miami Heat,Miami Heat,44,38,-0.32,0.18,-0.13,113.0,113.3,-0.3,96.3,0.270,0.408,0.574,0.530,12.4,22.8,0.224,0.561,14.5,77.7,0.198,0.0
14421,Cody Zeller,C,30,MIA,15,2,14.5,2.5,3.9,0.627,0.0,0.1,0.000,2.5,3.8,0.649,0.627,1.6,2.3,0.686,1.7,2.6,4.3,0.7,0.2,0.3,0.9,2.2,6.5,16.4,0.659,0.034,0.593,13.0,21.8,17.3,7.2,0.7,1.9,15.8,18.1,0.4,0.3,0.7,0.147,-2.0,-0.7,-2.8,0.0,

### Creating dataframe for DPOY

In [103]:
dpoy_subset = dpoy_df[['Player', 'Tm', 'Share', 'DRtg','Year']].copy()

dpoy_players_df = players_df.merge(dpoy_subset,
                                  how='left',
                                  left_on=['Player', 'Year', 'Tm'],
                                  right_on=['Player', 'Year', 'Tm'])

dpoy_players_df['Share'] = dpoy_players_df['Share'].fillna(0)
dpoy_players_df['DRtg'] = dpoy_players_df['DRtg'].fillna(0)
dpoy_players_df

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Year,Team_Long,Team,Team_W,Team_L,Team_MOV,Team_SOS,Team_SRS,Team_ORtg,Team_DRtg,Team_NRtg,Team_Pace,Team_FTr,Team_3PAr,Team_TS%,Team_eFG%,Team_TOV%,Team_ORB%,Team_FT/FGA,Team_eFG%.1,Team_TOV%.1,Team_DRB%,Team_FT/FGA.1,Share,DRtg
0,Tariq Abdul-Wahad,SG,25,DEN,61,56,25.9,4.5,10.6,0.424,0.0,0.4,0.130,4.4,10.2,0.435,0.426,2.4,3.2,0.756,1.7,3.1,4.8,1.6,1.0,0.5,1.7,2.4,11.4,13.6,0.477,0.036,0.299,7.0,13.1,10.0,10.0,1.8,1.2,12.7,22.5,0.4,1.8,2.2,0.068,-1.2,-0.1,-1.2,0.3,2000,Denver Nuggets,Denver Nuggets,35,47,-2.12,0.36,-1.76,103.5,105.8,-2.3,94.7,0.306,0.202,0.517,0.476,14.0,29.2,0.222,0.479,13.0,71.6,0.235,0.0,0.0
1,Tariq Abdul-Wahad,SG,25,ORL,46,46,26.2,4.8,11.2,0.433,0.0,0.5,0.095,4.8,10.7,0.447,0.435,2.5,3.3,0.762,1.7,3.5,5.2,1.6,1.2,0.3,1.9,2.5,12.2,14.4,0.484,0.041,0.293,7.0,14.6,10.8,9.7,2.2,0.9,13.0,23.3,0.4,1.6,2.0,0.082,-1.0,0.2,-0.8,0.4,2000,Orlando Magic,Orlando Magic,41,41,0.68,-0.26,0.43,102.4,101.7,0.7,97.3,0.305,0.124,0.516,0.473,15.4,31.6,0.224,0.476,15.9,69.9,0.226,0.0,0.0
2,Tariq Abdul-Wahad,SG,25,DEN,15,10,24.9,3.4,8.7,0.389,0.1,0.1,0.500,3.3,8.6,0.388,0.393,2.1,2.8,0.738,1.6,1.9,3.5,1.7,0.4,0.8,1.3,2.1,8.9,10.8,0.448,0.015,0.321,6.9,8.2,7.6,11.0,0.8,2.2,11.3,19.7,0.0,0.2,0.2,0.023,-1.7,-0.9,-2.5,0.0,2000,Denver Nuggets,Denver Nuggets,35,47,-2.12,0.36,-1.76,103.5,105.8,-2.3,94.7,0.306,0.202,0.517,0.476,14.0,29.2,0.222,0.479,13.0,71.6,0.235,0.0,0.0
3,Shareef Abdur-Rahim,SF,23,VAN,82,82,39.3,7.2,15.6,0.465,0.4,1.2,0.302,6.9,14.4,0.478,0.477,5.4,6.7,0.809,2.7,7.4,10.1,3.3,1.1,1.1,3.0,3.0,20.3,20.2,0.547,0.075,0.431,8.0,22.7,15.3,15.5,1.5,1.9,14.1,25.0,6.2,2.6,8.8,0.132,2.6,-0.4,2.2,3.4,2000,Vancouver Grizzlies,Vancouver Grizzlies,22,60,-5.62,0.52,-5.10,102.3,108.5,-6.2,91.0,0.320,0.139,0.524,0.474,15.8,30.1,0.247,0.502,14.1,70.6,0.231,0.0,0.0
4,Cory Alexander,PG,26,DEN,29,2,11.3,1.0,3.4,0.286,0.3,1.2,0.257,0.7,2.2,0.302,0.332,0.6,0.8,0.773,0.3,1.2,1.4,2.0,0.8,0.1,1.0,1.3,2.8,8.8,0.381,0.357,0.224,2.6,11.3,6.9,25.7,3.7,0.4,20.6,18.0,-0.5,0.4,-0.1,-0.012,-4.1,1.8,-2.4,0.0,2000,Denver Nuggets,Denver Nuggets,35,47,-2.12,0.36,-1.76,103.5,105.8,-2.3,94.7,0.306,0.202,0.517,0.476,14.0,29.2,0.222,0.479,13.0,71.6,0.235,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14418,Thaddeus Young,PF,34,TOR,54,9,14.7,2.0,3.7,0.545,0.1,0.6,0.176,1.9,3.0,0.622,0.561,0.3,0.5,0.692,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4,14.1,0.573,0.172,0.131,9.4,14.6,11.8,12.9,3.4,0.6,16.7,13.5,0.7,1.1,1.8,0.109,-1.8,1.9,0.1,0.4,2023,Toronto Raptors,Toronto Raptors,41,41,1.48,0.12,1.59,115.5,114.0,1.5,97.1,0.257,0.351,0.555,0.517,10.3,27.8,0.201,0.565,15.3,76.7,0.223,0.0,0.0
14419,Trae Young,PG,24,ATL,73,73,34.8,8.2,19.0,0.429,2.1,6.3,0.335,6.1,12.7,0.476,0.485,7.8,8.8,0.886,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2,22.0,0.573,0.331,0.460,2.4,7.0,4.7,42.5,1.5,0.3,15.2,32.6,5.3,1.4,6.7,0.126,5.3,-2.0,3.3,3.4,2023,Atlanta Hawks,Atlanta Hawks,41,41,0.29,0.02,0.32,116.6,116.3,0.3,100.7,0.244,0.331,0.579,0.541,11.2,25.1,0.200,0.552,12.4,75.8,0.206,0.0,0.0
14420,Omer Yurtseven,C,24,MIA,9,0,9.2,1.8,3.0,0.593,0.3,0.8,0.429,1.4,2.2,0.650,0.648,0.6,0.7,0.833,0.9,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4,16.7,0.675,0.259,0.222,10.9,21.9,16.2,3.9,1.2,2.5,11.9,18.0,0.2,0.1,0.3,0.159,-2.5,-1.5,-3.9,0.0,2023,Miami Heat,Miami Heat,44,38,-0.32,0.18,-0.13,113.0,113.3,-0.3,96.3,0.270,0.408,0.574,0.530,12.4,22.8,0.224,0.561,14.5,77.7,0.198,0.0,0.0
14421,Cody Zeller,C,30,MIA,15,2,14.5,2.5,3.9,0.627,0.0,0.1,0.000,2.5,3.8,0.649,0.627,1.6,2.3,0.686,1.7,2.6,4.3,0.7,0.2,0.3,0.9,2.2,6.5,16.4,0.659,0.034,0.593,13.0,21.8,17.3,7.2,0.7,1.9,15.8,1

### Creating dataframe for SMOY

In [104]:
smoy_subset = smoy_df[['Player', 'Tm', 'Share', 'Year']].copy()

smoy_players_df = players_df.merge(smoy_subset,
                                  how='left',
                                  left_on=['Player', 'Year', 'Tm'],
                                  right_on=['Player', 'Year', 'Tm'])

smoy_players_df['Share'] = smoy_players_df['Share'].fillna(0)
smoy_players_df

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Year,Team_Long,Team,Team_W,Team_L,Team_MOV,Team_SOS,Team_SRS,Team_ORtg,Team_DRtg,Team_NRtg,Team_Pace,Team_FTr,Team_3PAr,Team_TS%,Team_eFG%,Team_TOV%,Team_ORB%,Team_FT/FGA,Team_eFG%.1,Team_TOV%.1,Team_DRB%,Team_FT/FGA.1,Share
0,Tariq Abdul-Wahad,SG,25,DEN,61,56,25.9,4.5,10.6,0.424,0.0,0.4,0.130,4.4,10.2,0.435,0.426,2.4,3.2,0.756,1.7,3.1,4.8,1.6,1.0,0.5,1.7,2.4,11.4,13.6,0.477,0.036,0.299,7.0,13.1,10.0,10.0,1.8,1.2,12.7,22.5,0.4,1.8,2.2,0.068,-1.2,-0.1,-1.2,0.3,2000,Denver Nuggets,Denver Nuggets,35,47,-2.12,0.36,-1.76,103.5,105.8,-2.3,94.7,0.306,0.202,0.517,0.476,14.0,29.2,0.222,0.479,13.0,71.6,0.235,0.0
1,Tariq Abdul-Wahad,SG,25,ORL,46,46,26.2,4.8,11.2,0.433,0.0,0.5,0.095,4.8,10.7,0.447,0.435,2.5,3.3,0.762,1.7,3.5,5.2,1.6,1.2,0.3,1.9,2.5,12.2,14.4,0.484,0.041,0.293,7.0,14.6,10.8,9.7,2.2,0.9,13.0,23.3,0.4,1.6,2.0,0.082,-1.0,0.2,-0.8,0.4,2000,Orlando Magic,Orlando Magic,41,41,0.68,-0.26,0.43,102.4,101.7,0.7,97.3,0.305,0.124,0.516,0.473,15.4,31.6,0.224,0.476,15.9,69.9,0.226,0.0
2,Tariq Abdul-Wahad,SG,25,DEN,15,10,24.9,3.4,8.7,0.389,0.1,0.1,0.500,3.3,8.6,0.388,0.393,2.1,2.8,0.738,1.6,1.9,3.5,1.7,0.4,0.8,1.3,2.1,8.9,10.8,0.448,0.015,0.321,6.9,8.2,7.6,11.0,0.8,2.2,11.3,19.7,0.0,0.2,0.2,0.023,-1.7,-0.9,-2.5,0.0,2000,Denver Nuggets,Denver Nuggets,35,47,-2.12,0.36,-1.76,103.5,105.8,-2.3,94.7,0.306,0.202,0.517,0.476,14.0,29.2,0.222,0.479,13.0,71.6,0.235,0.0
3,Shareef Abdur-Rahim,SF,23,VAN,82,82,39.3,7.2,15.6,0.465,0.4,1.2,0.302,6.9,14.4,0.478,0.477,5.4,6.7,0.809,2.7,7.4,10.1,3.3,1.1,1.1,3.0,3.0,20.3,20.2,0.547,0.075,0.431,8.0,22.7,15.3,15.5,1.5,1.9,14.1,25.0,6.2,2.6,8.8,0.132,2.6,-0.4,2.2,3.4,2000,Vancouver Grizzlies,Vancouver Grizzlies,22,60,-5.62,0.52,-5.10,102.3,108.5,-6.2,91.0,0.320,0.139,0.524,0.474,15.8,30.1,0.247,0.502,14.1,70.6,0.231,0.0
4,Cory Alexander,PG,26,DEN,29,2,11.3,1.0,3.4,0.286,0.3,1.2,0.257,0.7,2.2,0.302,0.332,0.6,0.8,0.773,0.3,1.2,1.4,2.0,0.8,0.1,1.0,1.3,2.8,8.8,0.381,0.357,0.224,2.6,11.3,6.9,25.7,3.7,0.4,20.6,18.0,-0.5,0.4,-0.1,-0.012,-4.1,1.8,-2.4,0.0,2000,Denver Nuggets,Denver Nuggets,35,47,-2.12,0.36,-1.76,103.5,105.8,-2.3,94.7,0.306,0.202,0.517,0.476,14.0,29.2,0.222,0.479,13.0,71.6,0.235,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14418,Thaddeus Young,PF,34,TOR,54,9,14.7,2.0,3.7,0.545,0.1,0.6,0.176,1.9,3.0,0.622,0.561,0.3,0.5,0.692,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4,14.1,0.573,0.172,0.131,9.4,14.6,11.8,12.9,3.4,0.6,16.7,13.5,0.7,1.1,1.8,0.109,-1.8,1.9,0.1,0.4,2023,Toronto Raptors,Toronto Raptors,41,41,1.48,0.12,1.59,115.5,114.0,1.5,97.1,0.257,0.351,0.555,0.517,10.3,27.8,0.201,0.565,15.3,76.7,0.223,0.0
14419,Trae Young,PG,24,ATL,73,73,34.8,8.2,19.0,0.429,2.1,6.3,0.335,6.1,12.7,0.476,0.485,7.8,8.8,0.886,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2,22.0,0.573,0.331,0.460,2.4,7.0,4.7,42.5,1.5,0.3,15.2,32.6,5.3,1.4,6.7,0.126,5.3,-2.0,3.3,3.4,2023,Atlanta Hawks,Atlanta Hawks,41,41,0.29,0.02,0.32,116.6,116.3,0.3,100.7,0.244,0.331,0.579,0.541,11.2,25.1,0.200,0.552,12.4,75.8,0.206,0.0
14420,Omer Yurtseven,C,24,MIA,9,0,9.2,1.8,3.0,0.593,0.3,0.8,0.429,1.4,2.2,0.650,0.648,0.6,0.7,0.833,0.9,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4,16.7,0.675,0.259,0.222,10.9,21.9,16.2,3.9,1.2,2.5,11.9,18.0,0.2,0.1,0.3,0.159,-2.5,-1.5,-3.9,0.0,2023,Miami Heat,Miami Heat,44,38,-0.32,0.18,-0.13,113.0,113.3,-0.3,96.3,0.270,0.408,0.574,0.530,12.4,22.8,0.224,0.561,14.5,77.7,0.198,0.0
14421,Cody Zeller,C,30,MIA,15,2,14.5,2.5,3.9,0.627,0.0,0.1,0.000,2.5,3.8,0.649,0.627,1.6,2.3,0.686,1.7,2.6,4.3,0.7,0.2,0.3,0.9,2.2,6.5,16.4,0.659,0.034,0.593,13.0,21.8,17.3,7.2,0.7,1.9,15.8,18.1,0.4,0.3,0.7,0.147,-2.0,-0.7,-2.8,0.0,

### Creating a dataframe for ROTY

In [102]:
roy_subset = roy_df[['Player', 'Tm', 'Share', 'Year']].copy()

roy_players_df = players_df.merge(roy_subset,
                                  how='left',
                                  left_on=['Player', 'Year', 'Tm'],
                                  right_on=['Player', 'Year', 'Tm'])

roy_players_df['Share'] = roy_players_df['Share'].fillna(0)
roy_players_df

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Year,Team_Long,Team,Team_W,Team_L,Team_MOV,Team_SOS,Team_SRS,Team_ORtg,Team_DRtg,Team_NRtg,Team_Pace,Team_FTr,Team_3PAr,Team_TS%,Team_eFG%,Team_TOV%,Team_ORB%,Team_FT/FGA,Team_eFG%.1,Team_TOV%.1,Team_DRB%,Team_FT/FGA.1,Share
0,Tariq Abdul-Wahad,SG,25,DEN,61,56,25.9,4.5,10.6,0.424,0.0,0.4,0.130,4.4,10.2,0.435,0.426,2.4,3.2,0.756,1.7,3.1,4.8,1.6,1.0,0.5,1.7,2.4,11.4,13.6,0.477,0.036,0.299,7.0,13.1,10.0,10.0,1.8,1.2,12.7,22.5,0.4,1.8,2.2,0.068,-1.2,-0.1,-1.2,0.3,2000,Denver Nuggets,Denver Nuggets,35,47,-2.12,0.36,-1.76,103.5,105.8,-2.3,94.7,0.306,0.202,0.517,0.476,14.0,29.2,0.222,0.479,13.0,71.6,0.235,0.0
1,Tariq Abdul-Wahad,SG,25,ORL,46,46,26.2,4.8,11.2,0.433,0.0,0.5,0.095,4.8,10.7,0.447,0.435,2.5,3.3,0.762,1.7,3.5,5.2,1.6,1.2,0.3,1.9,2.5,12.2,14.4,0.484,0.041,0.293,7.0,14.6,10.8,9.7,2.2,0.9,13.0,23.3,0.4,1.6,2.0,0.082,-1.0,0.2,-0.8,0.4,2000,Orlando Magic,Orlando Magic,41,41,0.68,-0.26,0.43,102.4,101.7,0.7,97.3,0.305,0.124,0.516,0.473,15.4,31.6,0.224,0.476,15.9,69.9,0.226,0.0
2,Tariq Abdul-Wahad,SG,25,DEN,15,10,24.9,3.4,8.7,0.389,0.1,0.1,0.500,3.3,8.6,0.388,0.393,2.1,2.8,0.738,1.6,1.9,3.5,1.7,0.4,0.8,1.3,2.1,8.9,10.8,0.448,0.015,0.321,6.9,8.2,7.6,11.0,0.8,2.2,11.3,19.7,0.0,0.2,0.2,0.023,-1.7,-0.9,-2.5,0.0,2000,Denver Nuggets,Denver Nuggets,35,47,-2.12,0.36,-1.76,103.5,105.8,-2.3,94.7,0.306,0.202,0.517,0.476,14.0,29.2,0.222,0.479,13.0,71.6,0.235,0.0
3,Shareef Abdur-Rahim,SF,23,VAN,82,82,39.3,7.2,15.6,0.465,0.4,1.2,0.302,6.9,14.4,0.478,0.477,5.4,6.7,0.809,2.7,7.4,10.1,3.3,1.1,1.1,3.0,3.0,20.3,20.2,0.547,0.075,0.431,8.0,22.7,15.3,15.5,1.5,1.9,14.1,25.0,6.2,2.6,8.8,0.132,2.6,-0.4,2.2,3.4,2000,Vancouver Grizzlies,Vancouver Grizzlies,22,60,-5.62,0.52,-5.10,102.3,108.5,-6.2,91.0,0.320,0.139,0.524,0.474,15.8,30.1,0.247,0.502,14.1,70.6,0.231,0.0
4,Cory Alexander,PG,26,DEN,29,2,11.3,1.0,3.4,0.286,0.3,1.2,0.257,0.7,2.2,0.302,0.332,0.6,0.8,0.773,0.3,1.2,1.4,2.0,0.8,0.1,1.0,1.3,2.8,8.8,0.381,0.357,0.224,2.6,11.3,6.9,25.7,3.7,0.4,20.6,18.0,-0.5,0.4,-0.1,-0.012,-4.1,1.8,-2.4,0.0,2000,Denver Nuggets,Denver Nuggets,35,47,-2.12,0.36,-1.76,103.5,105.8,-2.3,94.7,0.306,0.202,0.517,0.476,14.0,29.2,0.222,0.479,13.0,71.6,0.235,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14418,Thaddeus Young,PF,34,TOR,54,9,14.7,2.0,3.7,0.545,0.1,0.6,0.176,1.9,3.0,0.622,0.561,0.3,0.5,0.692,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4,14.1,0.573,0.172,0.131,9.4,14.6,11.8,12.9,3.4,0.6,16.7,13.5,0.7,1.1,1.8,0.109,-1.8,1.9,0.1,0.4,2023,Toronto Raptors,Toronto Raptors,41,41,1.48,0.12,1.59,115.5,114.0,1.5,97.1,0.257,0.351,0.555,0.517,10.3,27.8,0.201,0.565,15.3,76.7,0.223,0.0
14419,Trae Young,PG,24,ATL,73,73,34.8,8.2,19.0,0.429,2.1,6.3,0.335,6.1,12.7,0.476,0.485,7.8,8.8,0.886,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2,22.0,0.573,0.331,0.460,2.4,7.0,4.7,42.5,1.5,0.3,15.2,32.6,5.3,1.4,6.7,0.126,5.3,-2.0,3.3,3.4,2023,Atlanta Hawks,Atlanta Hawks,41,41,0.29,0.02,0.32,116.6,116.3,0.3,100.7,0.244,0.331,0.579,0.541,11.2,25.1,0.200,0.552,12.4,75.8,0.206,0.0
14420,Omer Yurtseven,C,24,MIA,9,0,9.2,1.8,3.0,0.593,0.3,0.8,0.429,1.4,2.2,0.650,0.648,0.6,0.7,0.833,0.9,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4,16.7,0.675,0.259,0.222,10.9,21.9,16.2,3.9,1.2,2.5,11.9,18.0,0.2,0.1,0.3,0.159,-2.5,-1.5,-3.9,0.0,2023,Miami Heat,Miami Heat,44,38,-0.32,0.18,-0.13,113.0,113.3,-0.3,96.3,0.270,0.408,0.574,0.530,12.4,22.8,0.224,0.561,14.5,77.7,0.198,0.0
14421,Cody Zeller,C,30,MIA,15,2,14.5,2.5,3.9,0.627,0.0,0.1,0.000,2.5,3.8,0.649,0.627,1.6,2.3,0.686,1.7,2.6,4.3,0.7,0.2,0.3,0.9,2.2,6.5,16.4,0.659,0.034,0.593,13.0,21.8,17.3,7.2,0.7,1.9,15.8,18.1,0.4,0.3,0.7,0.147,-2.0,-0.7,-2.8,0.0,

### TO DO:

- [x] Convert rank column to a numeric in all of the award dataframes
- [ ] Filter each award dataframe by the relative minimum criteria to reduce data and create a more balanced dataset
- [ ] Merge player data with each awards table
- [ ] Perform data cleaning
- [ ] Plot distributions
- [ ] Analyze correlation
- [ ] Plot statistics